In [2]:
import os

os.chdir(r'D:\Desktop\MachineLearning\NLP\NLP新闻分类赛')
os.getcwd()

'D:\\Desktop\\MachineLearning\\NLP\\NLP新闻分类赛'

In [3]:
import pandas as pd
from sklearn.metrics import f1_score

# 转换为FastText需要的格式
train_df = pd.read_csv('./data/train_set.csv', sep='\t')
test_df = pd.read_csv('./data/test_a.csv', sep='\t')
# train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
# train_df[['text', 'label_ft']].to_csv('./preprocess/train_fastText.csv', index=None, header=None, sep='\t')

In [9]:
import fasttext

model = fasttext.train_supervised('./preprocess/train_fastText.csv',
                                  lr=0.01,
                                  epoch=50,
                                  dim=256,
                                  ws=5,
                                  seed=42,
                                  wordNgrams=3,
                                  verbose=2,
                                  minCount=3,
                                  t=1e-4,
                                  loss="softmax",)


In [14]:
model.save_model('./model/fastText_dim_256.bin')

In [5]:
import fasttext
model = fasttext.load_model('./model/fastText_dim_256.bin')

In [6]:
# for text in test_df['text']:
#     print(type(text))
#     print(model.predict("aaa"))
#     break;

<class 'str'>
(('__label__8',), array([1.00001001]))


In [7]:
# 预测测试集
# y_test_pred = []
# for text in test_df['text']:
#     pred_label = model.predict(text)[0][0].split('__')[-1]
#     y_test_pred.append(pred_label)
    
y_test_pred = [model.predict(text)[0][0].split('__')[-1] for text in test_df['text']]
# 将预测结果存储到 test_df 的 'label' 列
test_df['label'] = y_test_pred

# 保存结果到 CSV 文件
test_df['label'].to_csv("./result/submission_fastText.csv", index=False)
print("Results saved successfully.")

Results saved successfully.


In [ ]:
import fasttext
import numpy as np


def load_data(file_path):
    """
    加载数据
    """
    with open(file_path, 'r') as file:
        data = file.readlines()
    return data


def split_data(data, n_folds=5):
    """
    将数据划分为 n_folds 份
    """
    np.random.shuffle(data)
    fold_size = len(data) // n_folds
    folds = []
    for i in range(n_folds):
        start = i * fold_size
        end = (i + 1) * fold_size if i < n_folds - 1 else len(data)
        folds.append(data[start:end])
    return folds


def train_and_evaluate(folds, n_folds=5):
    """
    进行交叉验证
    """
    scores = []
    for i in range(n_folds):
        # 划分训练集和验证集
        validation_set = folds[i]
        train_set = [item for j, fold in enumerate(folds) if j!= i for item in fold]
        
        # 将训练集和验证集保存到临时文件
        with open('temp_train.txt', 'w') as train_file:
            train_file.writelines(train_set)
        with open('temp_val.txt', 'w') as val_file:
            val_file.writelines(validation_set)
        
        # 训练模型
        model = fasttext.train_supervised('temp_train.txt',
                                     lr=0.01,
                                     epoch=50,
                                     dim=256,
                                     ws=5,
                                     nthread=16,
                                     seed=42,
                                     wordNgrams=3,
                                     verbose=2,
                                     minCount=3,
                                     t=1e-4,
                                     loss="softmax")
        
        # 评估模型
        result = model.test('temp_val.txt')
        precision = result[1]
        scores.append(precision)
        print(f"Fold {i + 1} Precision: {precision}")
    
    return scores